In [ ]:
'''
This script reads the data_2013 directory and sequentially reads each MACnnnnn.csv file
and creates a summary dataset. The summary dataset is saved in the 'data_2013' directory
as 'summary.csv
'''
import pandas as pd
import numpy as np
import calendar

def get_lclid_data(lclid):
    df = pd.read_csv(
        f"data_2013/{lclid}.csv",
        usecols=lambda x: x != "LCLid",                                 # Drop LCLid column
        parse_dates=['DateTime'],                                       # Tell pandas how to read the date/time field
        date_format='%Y-%m-%d %H:%M:%S.%f0',
        dtype={'stdorToU': 'category'}
    )
    print("Processing LCLid: ", lclid)

    # Do some tidying up of the dataframe.
    df.rename(columns={df.columns[-1]: 'KWH'}, inplace=True)
    df['KWH'] = pd.to_numeric(df['KWH'], errors="coerce").astype(float) # convert KWH column to float
    df.drop_duplicates(inplace=True)                                    # drop all duplicate lines
    stdorToU = df['stdorToU'].mode()[0]                                 # Read most common stdorToU value
    df.drop(columns=["stdorToU"], inplace=True)                         # and then drop the column
    df.set_index('DateTime', inplace=True)
    # Note to self: May also need to check for any spurious datapoints that do not fall on the half-hour

    # Calculate a few statistics to add to the summary dataframe to give some indication of sample quality...

    #   Zero count: Number of provided rows with zero KWH
    zero_count_df = df[df['KWH'] == 0].count()
    zero_count = zero_count_df['KWH']

    #  Missing days: Number of dates of the year not in the data
    year = df.index[0].year                                             # Extract the year from the first index label
    num_days_in_year = df.index.dayofyear.nunique()                     # Num of days of data in dataset
    cal_year_days = 366 if calendar.isleap(year) else 365               # Num of days in calendar year
    missing_days = cal_year_days - num_days_in_year                     # Calc. Num of missing days

    # Incomplete days: Number of provided days with fewer than 48 halfhour readings
    daily_df = df['KWH'].groupby(df.index.dayofyear).count()            # Count num of KWH readings per day
    incomplete_days = daily_df[daily_df < 48].count()                   # Count num of days with fewer than 48 readings

    # Note that mean values for incomplete days will be included in the summary statistics
    # Also the number of missing and incomplete days are mutually exclusive
    # We do not keep track of the number of full/incomplete months but this could be done if needed

    # Note to self: An improvement here might be to start off with a so-called 'standard' dataframe which defines
    # every half hour period over the year as a NaN then fill in the rows as the csv is read in. This will
    # allow more dynamic data quality reporting without creating all these subframes.

    # Add summary statistics to a list
    summary_stats_list = [lclid, stdorToU, zero_count, missing_days, incomplete_days]

    # Calculate monthly summary statistics
    monthly_df = df.groupby(pd.Grouper(freq="ME")).mean()               # Group by month and calculate mean KWH
    monthly_df_array = [0] * 12                                         # Create an array of monthly mean values
    monthly_df.index = monthly_df.index.month.astype(int)               # Convert group index of month names to numbers

    for i, v in monthly_df.iterrows():                                  # Iterate over rows of monthly_df
        monthly_df_array[i-1] = v['KWH']                                # Update the array of monthly mean values

    return summary_stats_list + monthly_df_array                        # Put the two arrays together


column_list = ["LCLid","stdorToU", "zero_count", "missing_days", "incomplete_days", 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

idx_list = []
with open("data_2013/2013_ids.txt", "r") as input_file:
    for line in input_file:
        idx_list.append(line.strip())

summary_df = pd.DataFrame(index=idx_list, columns=column_list)

for lclid in idx_list:
    summary_df.loc[lclid] = get_lclid_data(lclid)

with open("data_2013/summary_2013.csv", "w") as output_file:
    summary_df.to_csv(output_file, lineterminator='\n')